# Useful to test syntaxes, data, structure before adding them to scripts

In [3]:
import warnings
warnings.simplefilter(action='ignore')

import functools
import argparse
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter
from scipy.stats import gamma
import os
import time
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchnet as tnt
from sklearn.neighbors import NearestNeighbors
import gc

import torch.nn as nn
from scipy.special import digamma, polygamma

import matplotlib
# Weird behavior: loading twice in cell appears to remove an elsewise occuring error.
for i in range(2):
    try:
        matplotlib.use('TkAgg')  # rerun this cell if an error occurs.
    except:
        print("!")
import matplotlib.pyplot as plt


import pickle
from torch_scatter import scatter_max, scatter_mean

print(torch.cuda.is_available())
np.random.seed(42)
torch.cuda.empty_cache()
print("Everything is imported")

False
Everything is imported


In [4]:
%matplotlib inline
import matplotlib.colors as clr

In [5]:
from osgeo import gdal
import sys
import os
sys.path.append("/home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/")
from config import args
from utils.useful_functions import create_new_experiment_folder, print_stats
# from data_loader.loader import normalize_cloud_data
# from utils.point_cloud_classifier import PointCloudClassifier
# from model.infer_utils import (
#     divide_parcel_las_and_get_disk_centers,
#     extract_points_within_disk,
#     create_geotiff_raster,
# )


# work with previous inference results
args.stats_path = "/home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/experiments/RESULTS_3_strata/only_stratum/DEV/inference/2021-06-08_15h55m34s/"

Dataset folder in use: /home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/data/
Arguments were imported in DEV mode


In [6]:
# create_new_experiment_folder(args)  # new paths are added to args

tiff_folder_path = os.path.join(args.stats_path, "img/rasters/004000715-5-18/")
input_file_list = os.path.join(tiff_folder_path, "rasters_paths.txt")

with open(input_file_list, "w") as f:
    for fname in os.listdir(tiff_folder_path):
        path = os.path.join(tiff_folder_path, fname)
        f.write(path)
        f.write("\n")

In [7]:
# GDAL python APIs doc: https://gdal.org/python/index.html
# PixelFunction in python - doc : https://gdal.org/drivers/raster/vrt.html#examples
# To add python function in vrt file: https://github.com/aerospaceresearch/DirectDemod/blob/1435456bbe4073faec2e812122be03e7094a582c/directdemod/merger.py#L29

tiff_paths = [
    os.path.join(tiff_folder_path, fname) for fname in os.listdir(tiff_folder_path)
]

vrt_raster_fname = os.path.join(tiff_folder_path, "merged_raster.vrt")
raster_fname = vrt_raster_fname.replace(".vrt",".tif")

vrt_options = gdal.BuildVRTOptions(
    resolution="highest",
    outputBounds=None,
    xRes=None,
    yRes=None,
    targetAlignedPixels=None,
    separate=None,
    bandList=None,
    addAlpha=None,
    resampleAlg=None,
    outputSRS=None,
    allowProjectionDifference=False,
    srcNodata=None,  # Carefull, perhaps to be specified
    VRTNodata=None,
    hideNodata=None,
    callback=None,
    callback_data=None,
)

vrt = gdal.BuildVRT(vrt_raster_fname, tiff_paths, options=vrt_options)
vrt.FlushCache()  # needed elsewise vrt not saved, see

In [8]:
"""inserts pixel-function 'average' into vrt file named 'filename'
Args:
    filename (:obj:`string`): name of file, into which the function will be inserted
    resample_name (:obj:`string`): name of resampling method
"""




pixelfunction_str ="""
import numpy as np
def average(in_ar, out_ar, xoff, yoff, xsize, ysize, raster_xsize,raster_ysize, buf_radius, gt, **kwargs):
    div = np.zeros(in_ar[0].shape)
    for i in range(len(in_ar)):
        div += (in_ar[i] != 0)
    div[div == 0] = 1
    
    y = np.sum(in_ar, axis = 0, dtype = 'uint16')
    y = y / div
    
    np.clip(y,0,255, out = out_ar)
"""


header = """  <VRTRasterBand dataType="Byte" band="1" subClass="VRTDerivedRasterBand">"""


contents = f"""
<PixelFunctionType>{"average"}</PixelFunctionType>
<PixelFunctionLanguage>Python</PixelFunctionLanguage>
<PixelFunctionCode><![CDATA[{pixelfunction_str}]]>
</PixelFunctionCode>"""

# rewrite
lines = open(vrt_raster_fname, 'r').readlines()
lines[3] = header  # FIX ME: 3 is a hand constant
lines.insert(4, contents)
with open(vrt_raster_fname, 'w') as f:
    f.write("".join(lines))

In [15]:
translated_options = gdal.TranslateOptions(
    options=None,
    format="GTiff",
    bandList=None,
    maskBand=None,
    width=0,
    height=0,
    widthPct=0.0,
    heightPct=0.0,
    xRes=0.0,
    yRes=0.0,
    creationOptions=None,
    srcWin=None,
    projWin=None,
    projWinSRS=None,
    strict=False,
    unscale=False,
    scaleParams=None,
    exponents=None,
    outputBounds=None,
    metadataOptions=None,
    outputSRS=None,
    nogcp=False,
    GCPs=None,
    noData=None,
    rgbExpand=None,
    stats=False,
    rat=True,
    resampleAlg=None,
    callback=None,
    callback_data=None,
)
vrt_ds = gdal.Open(vrt_raster_fname, gdal.GA_ReadOnly)
ds = gdal.Translate(raster_fname, vrt_ds, options=translated_options)
ds = None

In [13]:
# problème 1: ok si ds objetc mais pas si file !
os.path.isfile(vrt_raster_fname),os.path.isfile(raster_fname)

(True, False)

In [16]:
vrt_ds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7ff6fc42b6f0> >